In [1]:
import zipfile
import os

# 压缩文件路径
zip_path='./face.zip'
# 文件存储路径
save_path = './face/'

# 读取压缩文件
file=zipfile.ZipFile(zip_path)
# 解压文件
print('开始解压...')
file.extractall(save_path)
print('解压结束。')
# 关闭文件流
file.close()

开始解压...
解压结束。


In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
print(tf.test.is_gpu_available())

In [3]:
import os,sys
import numpy as np
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from PIL import Image
from sklearn.model_selection import train_test_split
import random
import cv2

In [ ]:
def DataSet():
    # 首先需要定义训练集和测试集的路径，这里创建了 train ， 和 test 文件夹
    train_path ='./face/rawdata/train/'
    test_path  ='./face/rawdata/test/'
    DR_sex_map = np.load("./face/DR_sex_map.npy")
    DS_sex_map = np.load("./face/DS_sex_map.npy")
    # os.listdir(path) 是 python 中的函数，它会列出 path 下的所有文件名
    # 比如说 imglist_train 对象就包括了/train 路径下所有的图片文件名
    imglist_train = os.listdir(train_path)
    #  读入标签
    
    # 下面两行代码读取了 /test 下的所有图片文件名
    imglist_test = os.listdir(test_path)
    # 这里定义两个 numpy 对象，X_train 和 Y_train
    
    # X_train 对象用来存放训练集的图片。每张图片都需要转换成 numpy 向量形式
    # X_train 的 shape 是 (360，224，224，3) 
    # 360 是训练集中图片的数量
    # 因为 resnet 要求输入的图片尺寸是 (224,224) , 所以要设置成相同大小（也可以设置成其它大小，参看 keras 的文档）
    # 1 是图片的通道数
    
    # Y_train 用来存放训练集中每张图片对应的标签
    # Y_train 的 shape 是 （1996，2）
    # 360 是训练集中图片的数量（训练集中固体胶和西瓜霜图片数量之和）
    # 因为一共有两种图片，所以第二个维度设置为 2
    # Y_train 大概是这样的数据 [[0,1],[0,1],[1,0],[0,1],...]
    # [0,1] 就是一张图片的标签，这里设置 [1,0] 代表male，[0,1] 代表female
    # 如果你有三类图片 Y_train 就因该设置为 (your_train_size,3)
    
    X_train = np.empty((len(DR_sex_map) , 224, 224, 3))
    Y_train = np.array([[1, 0] if i == 1 else [0, 1] for i in DR_sex_map])  
    # count 对象用来计数，每添加一张图片便加 1
    count = 0
    # 遍历 /train/glue 下所有图片，即训练集下所有的图片
    for img_name in imglist_train:
        # 得到图片的路径
        img_path = train_path + img_name
        with open(img_path, 'rb') as img_ori:  #将rawdata文件放在工作目录中
            img_data = np.frombuffer(img_ori.read(), dtype = np.uint8).reshape(128, 128)
            img_data = cv2.cvtColor(img_data, cv2.COLOR_GRAY2BGR)    #原图为灰度图，而dlib读取RGB图，故需要做格式转换
        img = cv2.resize(img_data,(224,224))/255.0#TODO
        # 将处理好的图片装进定义好的 X_train 对象中
        X_train[count] = img
        # 将对应的标签装进 Y_train 对象中，这里都是 固体胶（glue）图片，所以标签设为 [1,0]
        count+=1
          
    # 下面的代码是准备测试集的数据，与上面的内容完全相同，这里不再赘述
    X_test = np.empty((len(DS_sex_map) , 224, 224, 3))
    Y_test = np.array([[1, 0] if i == 1 else [0, 1] for i in DS_sex_map])  
    count = 0
    for img_name in imglist_test:
        with open(img_path, 'rb') as img_ori:  #将rawdata文件放在工作目录中
            img_data = np.frombuffer(img_ori.read(), dtype = np.uint8).reshape(128, 128)
            img_data = cv2.cvtColor(img_data, cv2.COLOR_GRAY2BGR)    #原图为灰度图，而dlib读取RGB图，故需要做格式转换
        img = cv2.resize(img_data,(224,224))/255.0#TODO
        X_test[count] = img
        count+=1
        
    # 打乱训练集中的数据
    index = [i for i in range(len(X_train))]
    random.shuffle(index)
    X_train = X_train[index]
    Y_train = Y_train[index]
    
    # 打乱测试集中的数据
    index = [i for i in range(len(X_test))]
    random.shuffle(index)
    X_test = X_test[index]    
    Y_test = Y_test[index]

    return X_train,Y_train,X_test,Y_test


X_train1,Y_train1,X_test1,Y_test1 = DataSet()
X = np.concatenate((X_train1,X_test1))
y = np.concatenate((Y_train1,Y_test1), axis=0)
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print('X_train shape : ',X_train.shape)
print('Y_train shape : ',Y_train.shape)
print('X_test shape : ',X_test.shape)
print('Y_test shape : ',Y_test.shape)

# # model
model = ResNet50(weights=None,classes=2)

model.compile(optimizer=tf.optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# # train

print('training.......')
model.fit(X_train, Y_train, epochs=40, batch_size=64)

# # evaluate

print('evaluating......')
model.evaluate(X_test, Y_test, batch_size=64)

# # save

print('saving......')
model.save('my_resnet_model.h5')

# # restore


model = tf.keras.models.load_model('my_resnet_model0.3.h5')

# # test

print('testing......')
with open("./face/rawdata/train/1223", 'rb') as img_ori:  #将rawdata文件放在工作目录中
    img_data = np.frombuffer(img_ori.read(), dtype = np.uint8).reshape(128, 128)
    img_data = cv2.cvtColor(img_data, cv2.COLOR_GRAY2BGR)    #原图为灰度图，而dlib读取RGB图，故需要做格式转换
img = cv2.resize(img_data,(224,224))
cv2.namedWindow('img1', cv2.WINDOW_FREERATIO|cv2.WINDOW_NORMAL)
cv2.imshow('img1', img)
cv2.waitKey(0)
cv2.destroyWindow('img1')
img = np.expand_dims(img, axis=0)  # 为batch添加第四维

print(model.predict(img))
np.argmax(model.predict(img))

X_train shape :  (1995, 224, 224, 3)
Y_train shape :  (1995, 2)
X_test shape :  (1996, 224, 224, 3)
Y_test shape :  (1996, 2)
training.......
Epoch 1/45
32/32 [==============================] - 185s 5s/step - loss: 1.4130 - accuracy: 0.5253
Epoch 2/45
32/32 [==============================] - 169s 5s/step - loss: 0.7491 - accuracy: 0.5454
Epoch 3/45
32/32 [==============================] - 169s 5s/step - loss: 0.7670 - accuracy: 0.5534
Epoch 4/45
32/32 [==============================] - 170s 5s/step - loss: 0.7140 - accuracy: 0.5454
Epoch 5/45
32/32 [==============================] - 169s 5s/step - loss: 0.7012 - accuracy: 0.5799
Epoch 6/45
32/32 [==============================] - 169s 5s/step - loss: 0.6975 - accuracy: 0.5609
Epoch 7/45
32/32 [==============================] - 169s 5s/step - loss: 0.7091 - accuracy: 0.5739
Epoch 8/45
32/32 [==============================] - 168s 5s/step - loss: 0.6864 - accuracy: 0.5960
Epoch 9/45
32/32 [==============================] - 169s 5s/step -